In [20]:
import chromadb
import logging
import os
import logging
import requests

schema_text = """
Table: users (id, name, email)
Table: orders (id, user_id, amount, date)
Table: products (id, name, price)
"""

In [21]:
def simple_chunk_lines(text, chunk_size=3):
    lines = text.strip().split('\n')
    chunks = []
    for i in range(0, len(lines), chunk_size):
        chunk = '\n'.join(lines[i:i+chunk_size]).strip()
        if chunk:
            chunks.append(chunk)
    return chunks
#WITH OVERLAP
# def chunk_with_overlap(text, chunk_size=3, overlap=1):
#     lines = text.strip().split('\n')
#     chunks = []
    
#     step = chunk_size - overlap  # Jump size is reduced to allow overlap
    
#     for i in range(0, len(lines), step):
#         chunk = '\n'.join(lines[i:i + chunk_size]).strip()
#         if chunk:
#             chunks.append(chunk)
            
#     return chunks


chunks = simple_chunk_lines(schema_text, chunk_size=1)


In [22]:


def get_embedding(text):
    OLLAMA_URL = "http://localhost:11434"
    EMBED_MODEL = "mxbai-embed-large"
    url = f"{OLLAMA_URL}/api/embeddings"
    payload = {"model": EMBED_MODEL, "prompt": text}

    response = requests.post(url, json=payload)
    response.raise_for_status()

    return response.json()["embedding"]

embeddings = [get_embedding(chunk) for chunk in chunks]


In [23]:
import chromadb

# # Connect to Chroma (new style)
# client = chromadb.PersistentClient(path="chroma_db")

# # Create or get collection
# collection = client.get_or_create_collection(name="sql_docs")

# # Suppose you have:
# # - chunks (already created)
# # - embeddings (you just generated)

# ids = [f"chunk_{i}" for i in range(len(chunks))]
# print(ids)
# collection.add(
#     documents=chunks,
#     embeddings=embeddings,
#     ids=ids
# )

# print("Embeddings successfully stored in ChromaDB!")


In [24]:
def StoreEmbeddingsInVec_db(embeddings, chunks):
    client = chromadb.PersistentClient(path="newchroma_db")
    collection = client.get_or_create_collection(name="sql_docs")
    ids = [f"chunk_{i}" for i in range(len(chunks))]
    
    collection.add(
    documents=chunks,
    embeddings=embeddings,
    ids=ids
    )
    print("Embeddings successfully stored in ChromaDB!")
    return collection
    

In [26]:
# user_question = "What is there"

# # Step 1: Get embedding for the question
# query_embedding = get_embedding(user_question)

# # Step 2: Query ChromaDB
# results = collection.query(
#     query_embeddings=[query_embedding],
#     n_results=3  # Top 3 most similar chunks
# )

# relevant_chunks = results['documents'][0]  # It's a list of lists

# # Combine them into a context
# context = "\n".join(relevant_chunks)

# print("Retrieved context:\n", context)

In [34]:
def retrieveContext(collection):
    userQuestion = "What is there"
    # Step 1: Get embedding for the question
    query_embedding = get_embedding(userQuestion)

# Step 2: Query ChromaDB
    results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3  # Top 3 most similar chunks
    )

    relevant_chunks = results['documents'][0]  # It's a list of lists

# Combine them into a context
    context = "\n".join(relevant_chunks)

    return context

    

In [28]:
# promptt = f"""
# You are an SQL expert. Based on the following schema and data chunks:
# {context}

# Answer the question:
# {user_question}
# """
def makePrompt(context, userQuestion):
    promptt = f"""
    You are an SQL expert. Based on the following schema and data chunks:
    {context}

    Answer the question:
    {userQuestion}
    """
    return promptt

In [29]:
import requests
import json
def ask_gemma(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {"model": "gemma3:1b", "prompt": prompt}

    response = requests.post(url, json=payload)
    response.raise_for_status()

    answer = ""
    for line in response.iter_lines():
        if line:
            data = line.decode('utf-8')
            json_obj = json.loads(data)   # Or use json.loads(data) if you're sure it's clean JSON
            if 'response' in json_obj:
                answer += json_obj['response']

    return answer




In [30]:
def Retrieve(text):
    chunks = simple_chunk_lines(text, chunk_size=1)
    embeddings = [get_embedding(chunk) for chunk in chunks]
    coll =StoreEmbeddingsInVec_db(embeddings, chunks)
    context = retrieveContext(coll)
    return context
    # prompt = makePrompt(context, query)
    # answer = ask_gemma(prompt)
    # return answer

In [31]:
def QueryLLM(question, context):
    prompt = makePrompt(context, question)
    answer = ask_gemma(prompt)
    return answer

In [32]:
context= Retrieve(schema_text)

Embeddings successfully stored in ChromaDB!


In [35]:
answer = QueryLLM("Hey, how youre doing?", context)
print(answer)

Okay, let's tackle this. You've provided a simple schema with a few tables representing products, users, and orders.  The question "Hey, how are you?" is a completely unrelated and nonsensical prompt. 

**My Response:**

As an SQL expert, I understand you've asked me a question that doesn't fit within the context of SQL.  My purpose is to help with data manipulation and analysis.  Since you’ve given me the table schema, I can offer some assistance if you need it.

**To help me answer your question effectively, could you please clarify *what* you're trying to achieve?**

For example, are you:

*   **Trying to write a query?** If so, what are you trying to accomplish? (e.g., "Give me all products with a price over $100")
*   **Looking for information about the tables?** (e.g., "Show me the columns in the `products` table")
*   **Need help with a specific task using SQL?** (e.g., "I need to join the `users` and `orders` tables to find orders placed by a specific user")


**I'm here to hel